In [1]:
import pandas as pd
import datetime
import numpy as np
import seaborn as sns
import requests, zipfile, io
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from xgboost import XGBRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from lightgbm import LGBMRegressor

/usr/local/lib/python3.7/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:17: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  "Since version 1.0, "


Note: I have not exposed the link to read dataset. Whoever wants to use this code, please reach out to me on my email(please check overivew page), so that i can share the dataset

Convert the Datetime column to as Number of Days from previous order

In [3]:
train_df['Last_order_placed_date'] = pd.to_datetime(train_df['Last_order_placed_date'])

train_df['Days_since_last_order'] = (datetime.datetime.now() - train_df['Last_order_placed_date']).dt.days

From EDA (check EDA file code), we get to know that the feature No_of_Orders_Placed and No_of_issue_raised are exactly same. So we can drop them

In [4]:
train_df = train_df.drop(columns=['Last_order_placed_date','Customer_ID','No_of_issues_raised'])

Assign the categorical and Ordinal columns to a numeric value

In [5]:
conversion_names = {
          "Category_of_customers" : {"Active" : 2, "Inactive":0,"Passive":1},
          "Premium_membership" : {"Yes" : 1, "No":0}
}
train_df.replace(conversion_names,inplace=True)

In [6]:
#separate the data into input features and output prediction
X = train_df.drop('Discount_percentage',axis=1)
y = train_df['Discount_percentage']

In [7]:
#split the data into train-test set(20% test data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

Below code contains three tree based - XGBOOST, HISTBOOSTER & LIGHTGBM

## XGBoost

In [8]:
#Build the model
xgbst_model = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:squarederror', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [9]:
#train the model on the train set
xgbst_model.fit(X_train,y_train)

XGBRegressor(max_depth=5, objective='reg:squarederror')

In [10]:
#Predict the output on both test & train set

yhat_xgbstval = xgbst_model.predict(X_test)

yhat_xgbsttrain = xgbst_model.predict(X_train)

In [11]:
#Predict the r2Score for both test & train
print("The score on the test set is:",r2_score(y_test,yhat_xgbstval)*100)

print("The score on the train set is:",r2_score(y_train,yhat_xgbsttrain)*100)

The score on the test set is: 53.23083453097005
The score on the train set is: 56.64031236884467


## HistBooster

In [12]:
#Build the model
histmodel = HistGradientBoostingRegressor(learning_rate=0.09)

In [13]:
#train the model on the train set
histmodel.fit(X_train,y_train)

HistGradientBoostingRegressor(learning_rate=0.09)

In [14]:
#Predict the output on both test & train set

yhist_val = histmodel.predict(X_test)

yhist_train = histmodel.predict(X_train)

In [15]:
#Predict the r2Score for both test & train
print("The score on the test set is:",r2_score(y_test,yhist_val)*100)

print("The score on the train set is:",r2_score(y_train,yhist_train)*100)

The score on the test set is: 53.47476147933061
The score on the train set is: 56.70578361751312


## LightGBM

In [16]:
#Build the model
lgbmodel = LGBMRegressor(learning_rate=0.09)

In [17]:
#Train the model on the train set
lgbmodel.fit(X_train,y_train)

LGBMRegressor(learning_rate=0.09)

In [18]:
#Predict the output on both test and train set

ylgb_val = lgbmodel.predict(X_test)

ylgb_train = lgbmodel.predict(X_train)

In [19]:
#Predict the r2Score for both test & train
print("The score on the test set is:",r2_score(y_test,ylgb_val)*100)

print("The score on the train set is:",r2_score(y_train,ylgb_train)*100)

The score on the test set is: 53.35156564777446
The score on the train set is: 58.01768506262549


## Averaging All three Models

In [20]:
#Averaging prediction on test and train set
pred_val =  (ylgb_val + yhist_val + yhat_xgbstval)/3

pred_train = (ylgb_train + yhist_train + yhat_xgbsttrain)/3

In [21]:
#Predict the final r2Score for both test & train
print("The score on the test set is:",r2_score(y_test,pred_val)*100)

print("The score on the train set is:",r2_score(y_train,pred_train)*100)

The score on the test set is: 53.4881633254732
The score on the train set is: 57.26144712595818


we observe that on averaging the models, the validation set or the test set accuracy improves. 